<a href="https://colab.research.google.com/github/Latiefabdul/Machine_Learning-With_TensorFlow/blob/main/Final_Exam%204%20%3A%20Natural_Language_Processing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
from tensorflow.keras import regularizers
import tensorflow.keras.utils as ku 
import numpy as np
import matplotlib.pyplot as plt

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
tokenizer = Tokenizer()
data = open('/content/drive/MyDrive/poem.txt').read()

corpus = data.lower().split("\n")


tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

# create input sequences using list of tokens
input_sequences = []
for line in corpus:
    token_list = tokenizer.texts_to_sequences([line])[0]
    for i in range(1, len(token_list)):
        n_gram_sequence = token_list[:i+1]
        input_sequences.append(n_gram_sequence)


# pad sequences 
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

# create predictors and label
predictors, label = input_sequences[:,:-1],input_sequences[:,-1]

label = ku.to_categorical(label, num_classes=total_words)

print(max_sequence_len)

16


In [4]:
#buatlah model dengan output layer total words/2 dan total word

model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150, return_sequences = True)))
model.add(Dropout(0.2))
model.add(LSTM(100))
model.add(Dense(total_words/2, activation='relu', kernel_regularizer=regularizers.l2(0.01)))
model.add(Dense(total_words, activation='softmax'))


model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 15, 100)           380800    
                                                                 
 bidirectional (Bidirectiona  (None, 15, 300)          301200    
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 15, 300)           0         
                                                                 
 lstm_1 (LSTM)               (None, 100)               160400    
                                                                 
 dense (Dense)               (None, 1904)              192304    
                                                                 
 dense_1 (Dense)             (None, 3808)              7254240   
                                                        

In [5]:
#Gunakan loss categorical_crossentropy
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [6]:
#training model 
history = model.fit(predictors, label, epochs=300, verbose=1)

Epoch 1/300
510/510 [==============================] - 88s 162ms/step - loss: 6.9675 - accuracy: 0.0612
Epoch 2/300
510/510 [==============================] - 82s 161ms/step - loss: 6.5281 - accuracy: 0.0628
Epoch 3/300
510/510 [==============================] - 82s 161ms/step - loss: 6.3628 - accuracy: 0.0649
Epoch 4/300
510/510 [==============================] - 86s 169ms/step - loss: 6.2447 - accuracy: 0.0681
Epoch 5/300
510/510 [==============================] - 82s 161ms/step - loss: 6.1629 - accuracy: 0.0744
Epoch 6/300
510/510 [==============================] - 83s 162ms/step - loss: 6.0969 - accuracy: 0.0776
Epoch 7/300
510/510 [==============================] - 82s 161ms/step - loss: 6.0277 - accuracy: 0.0812
Epoch 8/300
510/510 [==============================] - 86s 169ms/step - loss: 5.9375 - accuracy: 0.0890
Epoch 9/300
510/510 [==============================] - 82s 161ms/step - loss: 5.8400 - accuracy: 0.0967
Epoch 10/300
510/510 [==============================] - 82s 160m

In [7]:
#save model
model.save('model4.h5')